In [5]:
import math 

import numpy as np
import pandas as pd

import spacy
import spacy_fastlang

from rapidfuzz.fuzz import ratio

## Read data

In [32]:
df = pd.read_csv('../../data/tweets_with_spacy_lang.csv')

## Generate column with real language (if not exists)

In [33]:
if not "spacy_lang" in df.columns:
    nlp = spacy.load('es_core_news_sm')
    nlp.add_pipe("language_detector")

    df["spacy_lang"] = df.text.map(lambda x: nlp(x)._.language)
    df.to_csv("../../data/tweets_with_spacy_lang.csv", index=False)

## Process data

In [35]:
df.columns = df.columns.str.replace(" ", "_")
df["date"] = pd.to_datetime(df["date"])
df["is_rt"] = df["text"].str[:2] == "RT"
df["label"] = np.nan
df["id_tweet"] = "X1Y2Z3-" + df.id_tweet.astype(str)

## Split into files to translate

In [36]:
df_to_translate = (
    df
    .loc[
        (df.lang != "es") | (df.spacy_lang != "es"),
        ["id_tweet", "text"]
    ]
)

In [37]:
step = 15_000
size = df_to_translate.shape[0]
parts = math.ceil(size / step) 
dfs = [None] * parts 

print(f"{size} tweets to translate. Will split into {parts} parts of {step} tweets each.")
for i, min_ in enumerate(range(0, size, step)): 
    max_ = min_ + step
    if max_ > size:
        max_ = size              
    #print(min_, max_)
    dfs[i] = df_to_translate.iloc[min_:max_,]
    #print(dfs[i].shape[0])

134353 tweets to translate. Will split into 9 parts of 15000 tweets each.


In [40]:
ids = []
rows = []
for i, df_ in enumerate(dfs):
    rows.append(df_.shape[0])
    ids.extend(df_.id_tweet.tolist())
    df_.to_excel(f"../../data/translations/input/tweets_to_translate_{i}.xlsx", index=False)

In [41]:
assert sum(rows) == size
assert df.id_tweet[df.lang != "es"].isin(ids).all()
assert np.isin(ids, df.id_tweet.tolist()).all() 

Go to Google Translate and upload the files to translate, then continue with next step.

## Add translations to original dataset

In [42]:
all_translations = []
for i in range(parts):
    all_translations.append(pd.read_excel(f"../../data/translations/output/tweets_to_translate_{i}.xlsx"))

In [43]:
df_all_translations = pd.concat(all_translations)
df_all_translations.rename(columns={" texto": "texto_traducido"}, inplace=True)
df_all_translations["id_tweet"] = df_all_translations.id_tweet.str.strip()
df_all_translations["texto_traducido"] = df_all_translations.texto_traducido.str.strip()

In [44]:
df_all_translations.id_tweet.str.len().value_counts(dropna=False)

26    129659
25      4671
27        19
24         3
28         1
Name: id_tweet, dtype: int64

In [45]:
def get_most_similar_id(id_tweet):
    most_similar = None 
    most_similar_ratio = -1
    for id_comp in ids:
        ratio_ = ratio(id_tweet, id_comp, score_cutoff=90)
        if ratio_ > most_similar_ratio:
            most_similar = id_comp
            most_similar_ratio = ratio_
    return most_similar

In [46]:
fixed_tweet_ids = df_all_translations[~df_all_translations.id_tweet.isin(ids)].id_tweet.map(get_most_similar_id)
df_all_translations.loc[~df_all_translations.id_tweet.isin(ids), "id_tweet"] = fixed_tweet_ids

In [47]:
fixed_tweet_ids.shape

(58,)

In [48]:
assert df_all_translations.shape[0] == size
assert df_all_translations.id_tweet.isin(ids).all()
assert df[df.lang != "es"].id_tweet.isin(df_all_translations.id_tweet).all()
assert (df_all_translations.id_tweet.str[:6] == "X1Y2Z3").all()

### Merge datasets

In [49]:
df_full = df.merge(df_all_translations, how="left", on="id_tweet")
df_full["id_tweet"] = df_full.id_tweet.str[7:]

In [50]:
df_full.loc[(df_full.texto_traducido.isna()) & (df_full.lang != 'es') & (df_full.text.str.len() >= 30)]

,id_tweet,date,author,text,app,id_user,followers,following,stauses,location,...,user_retweeted,quoted_id,user_quoted,first_HT,lang,link,spacy_lang,is_rt,label,texto_traducido


In [51]:
df_full.to_csv("../../data/tweets_traducidos.csv", index=False)